# Aufgabe 2: Data Exploration und Analyse

In [ ]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import math

In [ ]:
# Datensatz einlesen
dataset = pd.read_csv('./DatenAusgegeben1.0.csv', delimiter=';', encoding = 'cp852')
dataset

Import seaborn und pyplot

In [ ]:
columns = ['Grundstück in qm', 'Zustand', 'Gebaut',
           'Renoviert', 'Zustand Fassade', 'Kellerflche in qm',
           'Erster Stock in qm', 'Zweiter Stock in qm', 'Wohnflche in qm', 
           'Schlafzimmer', 'Küchen', 'Küchenqualitt', 'Rume', 'Garagenkapazitt',
           'Verkaufsmonat', 'Verkaufsjahr', 'Preis']

## Finden von Anomalien
Zuerst müssen Ausreißer im Datensatz gefunden werden. Hierfür eignen sich Boxplots sehr gut.

In [ ]:
## Durch Subplots mehrere Plots auf einer Figure
fig, axes = plt.subplots(4, 5, figsize = (10, 15))
i = 0
plt.tight_layout(pad = 2)
for column in columns:
  chosenax = axes[math.floor(i / 5)][i % 5]
  chosenax.set_title(column)
  sb.boxplot(data = dataset[column], ax = chosenax)
  i = i + 1
plt.show()

Bei dem obenstehenden Diagramm zeigt sich, dass sich in einigen Spalten durchaus starke Ausreißer finden.  
Hier ist als Beispiel die Grundstücksgröße, Wohnfläche und den Preis zu nennen. Diese Ausreißer sollten in der Data Preparation folglich bereinigt werden

## Untersuchung auf einen Zusammenhang zwischen Preis und Verkaufszeitpunkt
Um zu Untersuchen ob der Zeitpunkt einen Einfluss auf den Verkaufspreis besitzt, untersuchen wir zwei Boxplots die den Zusammenhang zwischen Verkaufszeitpunkt und Verkaufspreis darstellen sollen.

In [ ]:
sb.boxplot(data = dataset[['Verkaufsjahr', 'Preis']], x = 'Verkaufsjahr', y = 'Preis')
plt.show()
sb.boxplot(data = dataset[['Verkaufsmonat', 'Preis']], x = 'Verkaufsmonat', y = 'Preis')
plt.show()

### Erkenntnis
Aus diesen zwei Diagrammen lässt sich kein Zusammenhang zwischen Zeitpunkt und Preis erkennen. Verkaufsjahr und Verkaufsmonat besitzen keinen Einfluss auf den Preis.

## Untersuchung des Datensatzes auf vernachlässigbare Hauseigenschaften (Spalten)

Da nicht-aussagekräftige Spalten nicht in unser Modell einfließen sollten, wird folgend der Datensatz auf vernachlässigbare bzw. nichtssagende Eigenschaften untersucht.

In [ ]:
attribute_columns = ['Grundstücksform', 'Steigung', 'Zone', 'Lage',
'Typ', 'Zustand', 'Zustand Fassade', 'Heizung',
'Heizungsqualitt', 'Klimaanlage', 'Schlafzimmer', 'Küchen',
'Küchenqualitt', 'Rume', 'Garage Typ', 'Garagenkapazitt', 'Pool'] # Alle Spalten die Immobilieneigenschaften repräsentieren

data_no_nan = dataset[attribute_columns].fillna('Keine') # Füllt leere Zeilen (bspw. bei Pool und GaragenTyp) mit einem Standardwert.

column_amount = len(data_no_nan.columns)
fig, axes = plt.subplots(math.ceil(column_amount / 4), 4, figsize = (15, 15))

i = 0
plt.tight_layout(pad = 3)
for column in data_no_nan.columns:
  chosenax = axes[math.floor(i / 4)][i % 4]
  chosenax.set_title(column)
  sb.countplot(data = data_no_nan, x = column, ax = chosenax)
  i = i + 1
plt.show()


### Erkenntnis
Hier hat sich herausgestellt, dass einige Spalten nicht aussagekräftig genug sind, da die Spalten eine zu geringe Varianz besitzen.

Spalten mit sehr geringer Varianz:
- Pool
- Küchen(anzahl)
- Klimaanlage
- Heizung



## Korrelation zwischen Preis und Immmobilieneigenschaften
Um die Korrelation visualisieren zu können wird folgend eine einspaltige Heatmap genutzt.  
Aus dieser soll sich herauslesen lassen können welche Attribute ausschlaggebend für einen höheren, oder auch einen niedrigeren Preis sind.

In [ ]:
# Ordinale Daten
plt.figure(figsize = (12, 10))
sb.heatmap(data = dataset.fillna(0).corr()['Preis'].to_frame(), annot = True, cmap = 'viridis')
plt.show()

In [ ]:
# Kategoriale Daten
encoded_kategorieeigenschaften = pd.get_dummies(dataset[['Typ', 'Steigung', 'Grundstücksform', 'Garage Typ', 'Klimaanlage', 'Zone', 'Pool', 'Heizung']].fillna('Keine'))

bezirk_preis = dataset[['Preis']].join(encoded_kategorieeigenschaften)
plt.figure(figsize = (12, 10))
sb.heatmap(data = bezirk_preis.corr()['Preis'].to_frame(), annot = True, cmap = 'viridis')
plt.show()

### Erkenntnis
In einer Korrelation mit dem Preis stehende Eigenschaften sind:
- Grundstücksgröße
- Baujahr
- Renovierungsjahr
- Wohnfläche (1. Stock + 2. Stock)
- Kellerfläche
- Küchenqualität
- Räume
- Garagenkapazität

Unter den Kategorialen Daten stehen mit dem Preis in Korrelation:
- Typ
- Grundstücksform
- Garagentyp

Trotz Korrelation aufgrund mangelnder Aussagekraft nicht beachtete Kategoriale Daten:
- Klimaanlage
- Verkaufsmonat
- Verkaufsjahr

## In welchem Preisumfeld liegt welcher Bezirk?
Um den Investoren eine Empfehlung zu geben mit welchem Budget sie in welchen Bezirk investieren sollten (Beantwortung des Ziels 3), sollen die Quartile der Hauspreise je Bezirk betrachtet werden.

Ebenso soll betrachtet werden ob die Auswahl eines einzelnen Bezirks im Vergleich zu allen anderen Bezirken eine Auswirkung auf den Verkaufspreis einer Immobilie besitzt.

In [ ]:
plt.figure(figsize = (15,8))
sb.boxplot(data = dataset, x = 'Preis', y = 'Bezirk')
plt.show()

In [ ]:
pd.get_dummies(dataset['Bezirk']).replace(0, math.nan).mul(dataset['Preis'], axis = 0).quantile([.25,.5,.75])

### Erkenntnis
Als Orientierung für das Budget der Investierenden kann die oben gezeigte Tabelle zu Rate gezogen werden.  
Das 0.25 Quartil sollte als Mindest-Budget und das 0.75 Quartil als obere Budgetgrenze gelten.

In [ ]:
encoded_bezirke = pd.get_dummies(dataset['Bezirk'])
bezirk_preis = dataset[['Preis']].join(encoded_bezirke)
plt.figure(figsize = (12, 10))
sb.heatmap(data = bezirk_preis.corr()['Preis'].to_frame(), annot = True, cmap = 'viridis').set_title('Korrelation zwischen Bezirk und Preis')
plt.show()

### Erkenntnis
Durch diese Heatmap lässt sich erkennen, wie die Auswahl eines Bezirkes eine Auswirkung auf den Preis einer Immobilie hat.  
Beispielsweise besitzen die Bezirke 'North East' und 'East End' mit einem Korrelationswert von +0,38 einen hohen positiven Einfluss auf dem Preis, d.h. Immobilien dort sind teurer als Beispielsweise in dem Bezirk 'Old Gotham' mit einem Korrelationswert von -0,2.